In [4]:
import os
from langchain_community.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.schema import  SystemMessage,HumanMessage,AIMessage 
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load API key from .env file
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or "YOUR_API_KEY"

# load json files
def load_jfile(file:list):
    folder_path = './data/'
    loader = JSONLoader(
        file_path=folder_path+file,
        jq_schema='.[]',
        content_key='html',
    )
    text_spilter = RecursiveCharacterTextSplitter(chunk_size=1000 , chunk_overlap=100)
    docs = text_spilter.split_documents(loader.load())
    return docs

class Chat_with_ai():
    def __init__(self,data=None,k=3):
        self.data = data
        self.query = ''
        self.k = k
        self.messages = [
            SystemMessage(content="You are a helpful assistant for cyber security."),
            SystemMessage(content="The transcript is the ATT&CK matrix. I need the solution to defense attcks and the reference with the transcript."),
            SystemMessage(content="**Reply in zh-tw**"),
        ]
        # data ==none continue to chat
        self.db = 'None'
        self.transcript = 'None'
        if data != None:
            self.db = self.data_to_db(data)
            self.transcript = self.get()
    
    def data_to_db(self,data)->FAISS:
        embedding = OpenAIEmbeddings()
        db = FAISS.from_documents(data,embedding)
        return db
    
    def get(self):
        docs =self.db.similarity_search(query = self.query, k = self.k)
        # print(docs)
        docs_page_content = ' '.join([str(doc) for doc in docs])
        return docs_page_content
    
    def chat(self):
        if self.query =='':
            return '請輸入查詢內容'
        
        model = ChatOpenAI(
            openai_api_key=os.environ["OPENAI_API_KEY"],
            model='gpt-3.5-turbo'
        )
        self.messages.append(SystemMessage(content=self.transcript))
        self.messages.append(HumanMessage(content=self.query))
        response =model.invoke(self.messages).content
        self.messages.append(AIMessage(content=response))
        return response


In [5]:
q ="""
Reconnaissance在ATT&CK框架中代表什麼？
列舉幾種Reconnaissance的常見技術或方法。
Resource Development在ATT&CK框架中是指什麼？
舉例說明資源開發可以如何被攻擊者利用來進一步入侵系統。
Initial Access在ATT&CK框架中的角色是什麼？
舉例說明一個Initial Access的攻擊技術。
在防禦中，如何可以減輕或阻止Reconnaissance階段的攻擊？
當懷疑系統可能已經受到Reconnaissance攻擊時，應該採取哪些措施？
Resource Development階段可能會使用哪些類型的資源？
列舉一些Initial Access的常見入侵點或攻擊向量。
"""
result = []
for i in q.split('\n'):
    if i:
        result.append(i)

print(result)

['Reconnaissance在ATT&CK框架中代表什麼？', '列舉幾種Reconnaissance的常見技術或方法。', 'Resource Development在ATT&CK框架中是指什麼？', '舉例說明資源開發可以如何被攻擊者利用來進一步入侵系統。', 'Initial Access在ATT&CK框架中的角色是什麼？', '舉例說明一個Initial Access的攻擊技術。', '在防禦中，如何可以減輕或阻止Reconnaissance階段的攻擊？', '當懷疑系統可能已經受到Reconnaissance攻擊時，應該採取哪些措施？', 'Resource Development階段可能會使用哪些類型的資源？', '列舉一些Initial Access的常見入侵點或攻擊向量。']


In [11]:
file = 'Reconnaissance.json'
j_data = load_jfile(file)
# pprint(j_data)

query = ['Reconnaissance在ATT&CK框架中代表什麼？', '列舉幾種Reconnaissance的常見技術或方法。', 'Resource Development在ATT&CK框架中是指什麼？', '舉例說明資源開發可以如何被攻擊者利用來進一步入侵系統。', 'Initial Access在ATT&CK框架中的角色是什麼？', '舉例說明一個Initial Access的攻擊技術。', '在防禦中，如何可以減輕或阻止Reconnaissance階段的攻擊？', '當懷疑系統可能已經受到Reconnaissance攻擊時，應該採取哪些措施？', 'Resource Development階段可能會使用哪些類型的資源？', '列舉一些Initial Access的常見入侵點或攻擊向量。']

model = Chat_with_ai(j_data)
model.query = query[0]
print(model.chat())

在ATT&CK框架中，Reconnaissance代表對目標進行情報收集的階段，包括搜尋目標的資訊、網路掃描、攻擊表面分析等活動。攻擊者通常在入侵前會進行Reconnaissance階段，以更好地了解目標並規劃後續的攻擊行動。


In [19]:
model.query = query[1]
print(model.chat())

Reconnaissance的常見技術或方法包括：

1. 網絡掃描：掃描目標網絡以識別主機、端口和服務。
2. 社交工程：通過偽造身份或利用社交媒體等方式，獲取目標組織的相關信息。
3. 電子郵件偵察：通過發送釣魚郵件或分析郵件頭部信息等方式，獲取目標組織的電子郵件架構和人員信息。
4. 開源情報收集：通過監視公開可用的信息來獲取對目標組織有用的情報。
5. 網站探測：通過訪問目標組織的網站或子域名，獲取有關組織結構和技術堆棧的信息。

這些技術和方法幫助駭客在攻擊前階段收集關鍵信息，以便制定更有針對性的攻擊策略。


In [20]:
model.query = query[2]
print(model.chat())

在ATT&CK框架中，Resource Development指的是駭客為進一步攻擊目標組織而收集、建立或獲取攻擊所需的資源、工具或設施的活動。這些資源可能包括惡意軟件、漏洞利用工具、社交工程技巧等，用於實施後續的攻擊階段，如交付、執行和持久化等。Resource Development旨在準備攻擊者在攻擊過程中所需的各種資源，以提高攻擊成功的機會。


In [ ]:
"""
Reconnaissance在攻擊生命週期中扮演什麼角色？
請解釋Reconnaissance的目的以及為什麼攻擊者會進行這個階段。
Reconnaissance階段包括哪些技術或方法？
舉例說明一個用於Reconnaissance的主動掃描技術。
攻擊者在Reconnaissance階段通常會收集哪些類型的資訊？
Reconnaissance階段的資訊可以用於什麼其他階段的攻擊？
在防禦方面，有哪些方法可以檢測或防止Reconnaissance攻擊？
Reconnaissance階段的目標是什麼？攻擊者通常希望從中獲得什麼樣的資訊？
舉例說明一個用於搜索封閉來源的Reconnaissance技術。
Reconnaissance階段的主要技術如何幫助攻擊者規劃未來的操作？
"""

In [13]:
model.query = '在防禦中，如何減輕或阻止Reconnaissance階段的攻擊？(附上transcript中的資料)'
print(model.chat())

根據transcript中的參考資料，以下是在防禦中減輕或阻止Reconnaissance階段攻擊的建議：

1. 監控網路流量和系統日誌，以及異常行為或可能的Reconnaissance活動。
2. 實施強化的身分驗證機制，防止未經授權的訪問。
3. 定期更新系統和應用程式，以修補潛在的漏洞，降低攻擊風險。
4. 教育員工辨識和報告可疑的電子郵件、網站或其他Reconnaissance活動。

這些措施有助於提高組織對於攻擊的警覺性，並降低受到Reconnaissance階段攻擊的風險。

參考資料：
- Babon, P. (2020, September 3). Tricky 'Forms' of Phishing. Retrieved October 20, 2020.
- Kan, M. (2019, October 24). Hackers Try to Phish United Nations Staffers With Fake Login Pages. Retrieved October 20, 2020.
- Nick Simonian. (2023, May 22). Don't @ Me: URL Obfuscation Through Schema Abuse. Retrieved August 4, 2023.
- NIST Information Technology Laboratory. (n.d.). web bug. Retrieved March 22, 2023.
- ZScaler. (2020, February 11). Fake Sites Stealing Steam Credentials. Retrieved March 8, 2023.
- mr.d0x. (2022, March 15). Browser In The Browser (BITB) Attack. Retrieved March 8, 2023.
- Proofpoint. (n.d.). The Human Factor 2023: Analyzing the cyber attack chain. Retrieved July 20, 2023.
- Dr. Nestori Syynimaa. (2018, October 25). A